In [1]:
import os 
import pandas as pd
import re
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import datetime, timedelta
from PIL import Image
import sys

Using TensorFlow backend.


In [2]:
img_width, img_height = 224, 224
batch_size = 1
model_path = '/data/ResNet50/ResNet50_20190404_species_weights.h5'

# Define the model
ResNet50 = ResNet50(weights=None, include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
model = Sequential()
model.add(ResNet50)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))

# Load the pretrained weights
model.load_weights(model_path)
model.summary()

/home/ubuntu/anaconda3/envs/tf_env/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                4112      
Total params: 49,282,192
Trainable params: 49,229,072
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
path = '/home/ubuntu/CreatureCounter/rest_api/test_folder/upload'

df = pd.DataFrame({'file':os.listdir(path)})
df['abs_file_path'] = df['file'].apply(lambda x: os.path.join(path, x))
df['category_name'] = 'unknown'

df

n_samples = len(df)

datagen = ImageDataGenerator(rescale=1/255.)

generator = datagen.flow_from_dataframe(
    df, 
    directory=path,
    x_col='file', 
    y_col ='category_name', 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    shuffle=False,
    class_mode='categorical')

predictions = model.predict_generator(
    generator, 
    steps=n_samples//batch_size,
    verbose=True
)

Found 3 images belonging to 1 classes.
3/3 [==============================] - 3s 1s/step


In [12]:
predictions.argmax([::5])

SyntaxError: invalid syntax (<ipython-input-12-78954ae4af8d>, line 1)

In [4]:
def decode_predictions(predictions):
    id_map = {
    0: 'american_black_bear',
    1: 'bobcat',
    2: 'cougar',
    3: 'coyote',
    4: 'domestic_cow',
    5: 'domestic_dog',
    6: 'elk',
    7: 'gray_fox',
    8: 'moose',
    9: 'mule_deer',
    10: 'red_deer',
    11: 'red_fox',
    12: 'vehicle',
    13: 'white_tailed_deer',
    14: 'wild_turkey',
    15: 'wolf'
}
    result = []
    for pred in predictions:
        result.append([(i, id_map[i], pred[i]) for i in pred.argsort()[-5:]])

    return result

In [10]:
data = {"requestSuccess": False}
data["predictions"] = []
data["requestTime"] = datetime.utcnow()

start = time.time()

# ensure an image was properly uploaded to our endpoint

path = '/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload'
data['userId'] = 'user'
base_path = os.path.dirname(path)
label_path = os.path.join(base_path,'classified')
wtf_path = os.path.join(base_path,'wtf')

file_df = pd.DataFrame({'file':os.listdir(path)})
file_df['abs_file_path'] = file_df['file'].apply(lambda x: os.path.join(path, x))
file_df['category_name'] = 'unknown'

n_samples = len(file_df)

datagen = ImageDataGenerator(rescale=1/255.)

generator = datagen.flow_from_dataframe(
    file_df, 
    directory=path,
    x_col='file', 
    y_col ='category_name', 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    shuffle=False,
    class_mode='categorical')

pred_start = time.time()

predictions = model.predict_generator(
    generator, 
    steps=n_samples,
    verbose=True
)

results = decode_predictions(predictions)

for i in range(len(results)):
    d = {'fileName':file_df['file'][i]}
    file_path = os.path.join(path, d['fileName'])
    d['imageId'] = "{:0.6f}".format((datetime.utcnow() - datetime(1970,1,1)).total_seconds()).replace('.', '_')
    d['fileSuccess'] = False
    r = [{"id":ID, "label": label, "probability": float(prob)} for (ID, label, prob) in results[i]]
    d['fileSuccess'] = True
    d['fileName'] = file_df['file'][i]
    d['imageId'] = "{:0.6f}".format((datetime.utcnow() - datetime(1970,1,1)).total_seconds()).replace('.', '_')
    d["labels"] = r
    d["procDatetime"] = datetime.utcnow()
    d["fileSuccess"] = True
    d["predictTime"] = time.time() - pred_start
    
    new_path = assign_new_path(r, 0.9, label_path, wtf_path)
    new_file_path = os.path.join(new_path, d['fileName'])
    print(file_path, new_path, new_file_path)
#     move_to_new_path(file_path, new_file_path)
    
    data['predictions'].append(d)

# # indicate that the request was a success
data["requestSuccess"] = True

# Send classification time
data["responseTime"] = time.time() - start

# # JSONify the response data
# response = flask.jsonify(data)

# If specified convert data to dataframe and push to database
sys.stdout.write('Converting data to dataframe')
sys.stdout.flush()
df = convert_to_df(data)

Found 8 images belonging to 1 classes.
9/9 [==============================] - 2s 212ms/step
/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/IMG_0143.jpg /home/ubuntu/CreatureCounter/rest_api/nacti_sample/wtf /home/ubuntu/CreatureCounter/rest_api/nacti_sample/wtf/IMG_0143.jpg
/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/PICT0737.JPG /home/ubuntu/CreatureCounter/rest_api/nacti_sample/classified/elk /home/ubuntu/CreatureCounter/rest_api/nacti_sample/classified/elk/PICT0737.JPG
/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/EK000016 (2).JPG /home/ubuntu/CreatureCounter/rest_api/nacti_sample/classified/red_deer /home/ubuntu/CreatureCounter/rest_api/nacti_sample/classified/red_deer/EK000016 (2).JPG
/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/EK000008 (2).jpg /home/ubuntu/CreatureCounter/rest_api/nacti_sample/wtf /home/ubuntu/CreatureCounter/rest_api/nacti_sample/wtf/EK000008 (2).jpg
/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/came

In [15]:
pd.DataFrame(data['predictions']).reset_index(drop=False)

,index,fileName,fileSuccess,imageId,labels,predictTime,procDatetime
0,0,IMG_0143.jpg,True,1555603966_296920,"[{'id': 0, 'label': 'american_black_bear', 'pr...",1.916070,2019-04-18 16:12:46.296924
1,1,PICT0737.JPG,True,1555603966_297101,"[{'id': 3, 'label': 'coyote', 'probability': 3...",1.916250,2019-04-18 16:12:46.297104
2,2,EK000016 (2).JPG,True,1555603966_297229,"[{'id': 0, 'label': 'american_black_bear', 'pr...",1.916378,2019-04-18 16:12:46.297232
3,3,EK000008 (2).jpg,True,1555603966_297348,"[{'id': 0, 'label': 'american_black_bear', 'pr...",1.916496,2019-04-18 16:12:46.297351
4,4,camera monitorring 2013 075.jpg,True,1555603966_297468,"[{'id': 9, 'label': 'mule_deer', 'probability'...",1.916617,2019-04-18 16:12:46.297471
5,5,Camera Monitoring August 2013 065.JPG,True,1555603966_297583,"[{'id': 0, 'label': 'american_black_bear', 'pr...",1.916731,2019-04-18 16:12:46.297586
6,6,.ipynb_checkpoints,True,1555603966_297698,"[{'id': 2, 'label': 'cougar', 'probability': 3...",1.916847,2019-04-18 16:12:46.297701
7,7,EK000709.JPG,True,1555603966_297813,"[{'id': 7, 'label': 'gray_fox', 'probability':...",1.916961,2019-04-18 16:12:46.297816
8,8,EK000020.jpg,True,1555603966_297949,"[{'id': 0, 'label': 'american_black_bear', 'pr...",1.917098,2019-04-18 16:12:46.297952


In [120]:
os.path.join(new_path, file_path)

'/home/ubuntu/CreatureCounter/rest_api/nacti_sample/upload/EK000020.jpg'

In [117]:
assign_new_path(r, 0.9, label_path, wtf_path)

'/home/ubuntu/CreatureCounter/rest_api/nacti_sample/classified/coyote'

In [7]:
# Determine which folder to move the predicted image to
def assign_new_path(labels, threshold, label_path, wtf_path):
    probs = [label["probability"] for label in labels]
    max_prob, max_id = max(probs), np.argmax(probs)
    pred = labels[max_id]["label"]
    if max_prob >= threshold:
        return os.path.join(label_path, pred)
    else:
        return wtf_path

# Move a file to a new folder
def move_to_new_path(old_file_path, new_file_path):
    # Get the new file path's directory
    new_folder = os.path.dirname(new_file_path)
    # If the new directory does not exist create it
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
    # Update the file path to the new file path
    os.rename(old_file_path, new_file_path)

In [102]:
predictions[0].argsort()[-5:]

array([ 3,  0,  4, 15, 12])

In [103]:
len(data['predictions'])

3

In [104]:
convert_to_df(data)

,fileName,fileSuccess,imageId,predictTime,procDatetime,imageNetId,label,probability,rank,userId
0,dog2.jpg,True,1555571482_792415,3751.953649,2019-04-18 07:11:22.792421,3,coyote,1.036272e-08,5.0,user
1,dog2.jpg,True,1555571482_792415,3751.953649,2019-04-18 07:11:22.792421,0,american_black_bear,1.348056e-07,4.0,user
2,dog2.jpg,True,1555571482_792415,3751.953649,2019-04-18 07:11:22.792421,4,domestic_cow,5.216637e-07,3.0,user
3,dog2.jpg,True,1555571482_792415,3751.953649,2019-04-18 07:11:22.792421,15,wolf,6.875128e-05,2.0,user
4,dog2.jpg,True,1555571482_792415,3751.953649,2019-04-18 07:11:22.792421,12,vehicle,9.999305e-01,1.0,user
5,dog1.jpg,True,1555571482_792510,3751.953742,2019-04-18 07:11:22.792515,3,coyote,1.036272e-08,5.0,user
6,dog1.jpg,True,1555571482_792510,3751.953742,2019-04-18 07:11:22.792515,0,american_black_bear,1.348056e-07,4.0,user
7,dog1.jpg,True,1555571482_792510,3751.953742,2019-04-18 07:11:22.792515,4,domestic_cow,5.216637e-07,3.0,user
8,dog1.jpg,True,1555571482_792510,3751.953742,2019-04-18 07:11:22.792515,15,wolf,6.875128e-05,2.0,user
9,dog1.jpg,True,1555571482_792510,3751.953742,2019-04-18 07:11:22.792515,12,vehicle,9.999305e-01,1.0,user


In [72]:
for idx, row in file_df.iterrows():
    pass

In [31]:
pred.argsort()

array([ 8,  7, 13,  9, 14, 11,  1,  5,  6, 10,  2,  3,  0,  4, 15, 12])

In [ ]:
    # loop over the results and add them to the list of
    # returned predictions
    r = [{"id":ID, "label": label, "probability": float(prob)} for (ID, label, prob) in results[0]]
    d["labels"] = r
    d["procDatetime"] = datetime.utcnow()
    d["fileSuccess"] = True
    d["predictTime"] = time.time() - pred_start

    data["predictions"].append(d)

    sys.stdout.write(str(d['fileName']) + ': ' + str(d['predictTime']))
    sys.stdout.flush()

    # Move the file to the correct folder
    new_path = assign_new_path(r, 0.9, label_path, wtf_path)
    new_file_path = os.path.join(new_path, file)
    move_to_new_path(file_path, new_file_path)

In [9]:
# Convert the Json output to a dataframe
def convert_to_df(label_json):
    label_df = pd.DataFrame(label_json['predictions'])
    pred_df = label_df[['imageId', 'labels']]
    pred_df.index = pred_df['imageId']
    pred_df = pred_df['labels'].apply(pd.Series).reset_index(drop=False).melt('imageId', value_vars=range(5))
    pred_df = pred_df.drop('variable', axis=1)
    pred_df['imageNetId'] = pred_df['value'].apply(lambda x: x['id'])
    pred_df['label'] = pred_df['value'].apply(lambda x: x['label'])
    pred_df['probability'] = pred_df['value'].apply(lambda x: x['probability'])
    pred_df = pred_df.drop('value', axis=1)
    pred_df['rank'] = pred_df.groupby('imageId')['probability'].rank(ascending=False)
    pred_df = pred_df.sort_values(['imageId', 'rank'], ascending=[True, False])

    label_df = label_df.merge(pred_df, on='imageId', how='left').drop('labels', axis=1)
    label_df['userId'] = label_json['userId']

    return label_df

In [ ]:
@app.route("/predictFolder", methods=["POST"])
def predict_folder():
    # initialize the data dictionary that will be returned from the
    # view
    start = time.time()

    data = {"requestSuccess": False}
    data["predictions"] = []
    data["requestTime"] = datetime.utcnow()
    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        payload = json.loads(flask.request.json)
        if payload.get("path"):
            # ensure an image was properly uploaded to our endpoint
            path = os.path.normpath(payload.get("path"))
            base_path = os.path.dirname(path)
            label_path = os.path.join(base_path,'classified')
            wtf_path = os.path.join(base_path,'wtf')

            data["userId"] = payload.get("userId")
            for file in os.listdir(path):
                # Note the point when the script started
                pred_start = time.time()

                file_path = os.path.join(path, file)

                image_id = "{:0.6f}".format((datetime.utcnow() - datetime(1970,1,1)).total_seconds()).replace('.', '_')
                d = {'fileName':file, 'imageId':image_id}
                d['fileSuccess'] = False

                # read the image in PIL format
                image = Image.open(os.path.join(path,file))

                # preprocess the image and prepare it for classification
                image = prepare_image(image, target=(224, 224))

                # classify the input image and then initialize the list
                # of predictions to return to the client
                with graph.as_default():
                    preds = model.predict(image)
                    
                print(preds)
                results = imagenet_utils.decode_predictions(preds)

                # loop over the results and add them to the list of
                # returned predictions
                r = [{"id":ID, "label": label, "probability": float(prob)} for (ID, label, prob) in results[0]]
                d["labels"] = r
                d["procDatetime"] = datetime.utcnow()
                d["fileSuccess"] = True
                d["predictTime"] = time.time() - pred_start

                data["predictions"].append(d)

                sys.stdout.write(str(d['fileName']) + ': ' + str(d['predictTime']))
                sys.stdout.flush()

                # Move the file to the correct folder
                new_path = assign_new_path(r, 0.9, label_path, wtf_path)
                new_file_path = os.path.join(new_path, file)
                move_to_new_path(file_path, new_file_path)

            # indicate that the request was a success
            data["requestSuccess"] = True

            # Send classification time
            data["responseTime"] = time.time() - start

    # JSONify the response data
    response = flask.jsonify(data)

    # If specified convert data to dataframe and push to database
    sys.stdout.write('Converting data to dataframe')
    sys.stdout.flush()
    df = convert_to_df(data)

    # Write output to the database
    sys.stdout.write('Writing dataframe to Postgres database')
    sys.stdout.flush()
    write_df_to_db(df, 'test_upload')

    # return the data dictionary as a JSON response
    return flask.jsonify(data)